In [2]:
#installing kaggle library
! pip install kaggle


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#api command
!kaggle datasets download -d bwandowando/3-million-instagram-google-store-reviews


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Kritika Pawar\AppData\Roaming\Python\Python313\Scripts\kaggle.exe\__main__.py", line 7, in <module>
    sys.exit(main())
             ~~~~^^
  File "C:\Users\Kritika Pawar\AppData\Roaming\Python\Python313\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
  File "C:\Users\Kritika Pawar\AppData\Roaming\Python\Python313\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Kritika Pawar\AppData\Roaming\Python\Python313\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [5]:
#extracting zip file
from zipfile import ZipFile
dataset = '/content/3-million-instagram-google-store-reviews.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/3-million-instagram-google-store-reviews.zip'

In [ ]:
#required libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

In [ ]:
instagram_data=pd.read_csv('/content/INSTAGRAM_REVIEWS.csv',encoding= 'ISO-8859-1')

In [ ]:
instagram_data.shape

In [ ]:
instagram_data.head()

In [ ]:
instagram_data.isnull().sum()

In [ ]:
instagram_data.dropna(subset=['review_text'], inplace=True)  # Drop missing reviews

instagram_data['author_app_version'] = instagram_data['author_app_version'].fillna('unknown') # Fill Nan Values

In [ ]:
instagram_data.isnull().sum()


In [ ]:
instagram_data.shape

In [ ]:
instagram_data.head()

In [ ]:
instagram_data['review_rating'].value_counts()

In [ ]:
#labelling the ratings
def label_sentiment(rating):
    if rating >= 4:
        return 'positive review'
    elif rating == 3:
        return 'neutral review'
    else:
        return 'negative review'

In [ ]:
instagram_data['sentiment'] = instagram_data['review_rating'].apply(label_sentiment)
print(instagram_data['sentiment'].value_counts())

In [ ]:
from sklearn.utils import resample

df_pos = instagram_data[instagram_data['sentiment'] == 'positive review']
df_neg = instagram_data[instagram_data['sentiment'] == 'negative review']
df_neu = instagram_data[instagram_data['sentiment'] == 'neutral review']

df_neg_upsampled = resample(df_neg,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_neu_upsampled = resample(df_neu,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_balanced = pd.concat([df_pos, df_neg_upsampled, df_neu_upsampled])
print(df_balanced['sentiment'].value_counts())


In [ ]:
port_stem=PorterStemmer()
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
instagram_data['stemmed_content']=instagram_data['review_text'].apply(stemming)

In [ ]:
instagram_data.head()

In [ ]:
# Clean missing stemmed_content first
instagram_data = instagram_data[instagram_data['stemmed_content'].notna()]
instagram_data = instagram_data[instagram_data['stemmed_content'].str.strip() != '']


In [ ]:
#separating data and label
X=instagram_data['stemmed_content'].values
Y=instagram_data['review_rating'].values

In [ ]:
from sklearn.utils import resample

df_pos = instagram_data[instagram_data['sentiment'] == 'positive review']
df_neg = instagram_data[instagram_data['sentiment'] == 'negative review']
df_neu = instagram_data[instagram_data['sentiment'] == 'neutral review']

df_neg_upsampled = resample(df_neg, replace=True, n_samples=len(df_pos), random_state=42)
df_neu_upsampled = resample(df_neu, replace=True, n_samples=len(df_pos), random_state=42)

df_balanced = pd.concat([df_pos, df_neg_upsampled, df_neu_upsampled])

# 4. Remove any missing or empty stemmed_content again (just in case)
df_balanced = df_balanced[df_balanced['stemmed_content'].notna()]
df_balanced = df_balanced[df_balanced['stemmed_content'].str.strip() != '']

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
#spliting data into test and training data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=20000)
X = vectorizer.fit_transform(instagram_data['stemmed_content'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df_balanced['sentiment'])


In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
print(instagram_data.shape)
print(instagram_data['review_rating'].shape)

print(df_balanced['sentiment'].shape)


In [ ]:
#training the model
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
#accuracy score on training model
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

In [ ]:
#accuracy score on test model
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

In [ ]:
baseline_accuracy = df_balanced['sentiment'].value_counts(normalize=True).max()
print(baseline_accuracy)


In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
#loading saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))


In [ ]:
X_new=X_test[55]
print(Y_test[55])
prediction= model.predict(X_new)
print(prediction)

if prediction[0] >= 4:
        print('positive review')
elif prediction[0] == 3:
        print('neutral review')
else:
        print('negative review')

In [ ]:
X_new=X_test[20]
print(Y_test[20])
prediction= model.predict(X_new)
print(prediction)

if prediction[0] >= 4:
        print('positive review')
elif prediction[0] == 3:
        print('neutral review')
else:
        print('negative review')